Estudiante: Arlex Fabian Galindez Rivera

In [1]:
!pip install colorama==0.4.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import random
from copy import deepcopy
import time
import math
from colorama import Fore, init

In [3]:
#Clase que tiene los metodos generales y el metodo para imprimir y cambiar la coordenada de la ficha
class Player(object):

  #Funcion encargada de aplicar el movimiento seleccionada por el jugador
  def aplica_movimiento(self,movimientos):
    return NotImplementedError

  #Funcion encargada de obtener los posibles movimientos que puede realizar el jugador
  def obtiene_movimientos(self):
    return NotImplementedError

  ##Funcion encargada de determinar si el juego termino y mostrar al ganador
  def es_estado_final(self):
    return NotImplementedError  

  ##Funcion encargada de mostrar al jugador ganador
  def gana_jugador(self,jugador):
    return NotImplementedError

  #Funcion encargada de evaluar el estado de juego para un jugador
  def heuristica(self,jugador):
    return NotImplementedError

  ##Funcion encargada de cambiar la coordenada de la ficha luego de un movimiento
  def cambio_coordenada(len,i):
    newI=len-1-i
    return newI
  
  #Funcion encargada de imprimir el movimiento seleccionado
  def imprime_movimientos(movimientos):
    i=0
    for movimiento in movimientos:
        if(len(movimiento)==4):
            print(i+1,"º movimiento:",movimiento)
        elif(len(movimiento)==2):
            print(i+1,"º movimiento:",movimiento[0],"con captura de ",movimiento[1])
        i+=1
  

In [8]:
#Clase encargada de todos las funciones para el desarrollo del juego
class Damas(Player):
    jugadores = 2

    def __init__(self):
        self.tablero = ([0,1,0,1,0,1,0,1],
                        [1,0,1,0,1,0,1,0],
                        [0,1,0,1,0,1,0,1],
                        [0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0],
                        [2,0,2,0,2,0,2,0],
                        [0,2,0,2,0,2,0,2],
                        [2,0,2,0,2,0,2,0])
        self.jugadorActual = 2
        self.turnos = 500
      
    #Funcion encargada de contar las fichas en el tablero y sus posiciones
    def heuristica(self,jugador):
        tablero = self.tablero
        contadorFichas=0
        contadorDamas=0
        contadorFichasEnemigas=0
        contadorDamasEnemigas=0
        for i in range(len(tablero)):
            for j in range(len(tablero)):
                if(tablero[i][j]==jugador):
                    contadorFichas+=1
                elif(tablero[i][j]==jugador+2):
                    contadorDamas+=1
                elif(tablero[i][j]==3-jugador):
                    contadorFichasEnemigas+=1
                elif(tablero[i][j]==3-jugador+2):
                    contadorDamasEnemigas+=1
        return 4*(contadorDamas-contadorDamasEnemigas)+(contadorFichas-contadorFichasEnemigas)

    #Funcion que comprueba si una ficha puede capturar algún rival y devuelve estos movimientos
    def comprobar_captura(self,i,j,lista,iInicial,jInicial,listaCapturas=[]):
        tablero=self.tablero
        jugador=self.jugadorActual
        if(jugador==1):
            peonRival=2
            reyRival=4
        else:
            peonRival=1
            reyRival=3
        der = j+1
        izq = j-1
        newI=Player.cambio_coordenada(len(tablero),i)
        if(jugador==1):
            arr=i+1
        else:
            arr=i-1
        if(der<=6 and (arr<=6 and arr>=1) and (tablero[arr][der]==peonRival or tablero[arr][der]==reyRival)):
            if(jugador==1):
                arr2=arr+1
            else:
                arr2=arr-1
            if(tablero[arr2][der+1]==0):
                newArr=Player.cambio_coordenada(len(tablero),arr)
                newListaCapturasDer=listaCapturas[:]
                newListaCapturasDer.append((der,newArr))
                self.comprobar_captura(arr2,der+1,lista,iInicial,jInicial,newListaCapturasDer)
        if(izq>=1 and (arr<=6 and arr>=1) and (tablero[arr][izq]==peonRival or tablero[arr][izq]==reyRival)):
            if(jugador==1):
                arr2=arr+1
            else:
                arr2=arr-1
            if(tablero[arr2][izq-1]==0):
                newArr=Player.cambio_coordenada(len(tablero),arr)
                newListaCapturasIzq=listaCapturas[:]
                newListaCapturasIzq.append((izq,newArr))
                self.comprobar_captura(arr2,izq-1,lista,iInicial,jInicial,newListaCapturasIzq)
        if(len(listaCapturas)>0):
            lista.append((len(listaCapturas),(jInicial,iInicial,j,newI),listaCapturas))
        return lista

    #Funcion que comprueba si una dama puede capturar y devuelve estos movimientos
    def comprobar_captura_dama(self,i,j,lista,iInicial,jInicial,listaCapturas=[]):
        tablero=self.tablero
        jugador=self.jugadorActual
        if(jugador==1):
            peonRival=2
            reyRival=4
        else:
            peonRival=1
            reyRival=3
        der = j+1
        izq = j-1
        newI=Player.cambio_coordenada(len(tablero),i)
        arr=i+1
        abj=i-1
        while(der<=6 and arr<=6):
            if(tablero[arr][der]==peonRival or tablero[arr][der]==reyRival):
                if(tablero[arr+1][der+1]==0):
                    newArr=Player.cambio_coordenada(len(tablero),arr)
                    newListaCapturasDer=listaCapturas[:]
                    newListaCapturasDer.append((der,newArr))
                    newEstado=deepcopy(self)
                    newEstado.tablero[arr][der]=0
                    newEstado.tablero[i][j]=0
                    newEstado.tablero[arr+1][der+1]=jugador
                    arr2=arr+1
                    der2=der+1
                    while(der2<=7 and arr2<=7):
                        if(tablero[arr2][der2]==0):
                            newEstado.comprobar_captura_dama(arr2,der2,lista,iInicial,jInicial,newListaCapturasDer)
                            der2=der2+1
                            arr2=arr2+1
                        else:
                            break
                    break
                else:
                    break
            elif(tablero[arr][der]==0):
                arr+=1
                der+=1
            else:
                break
        der=j+1
        abj=i-1
        while(der<=6 and abj>=1):
            if(tablero[abj][der]==peonRival or tablero[abj][der]==reyRival):
                if(tablero[abj-1][der+1]==0):
                    newArr=Player.cambio_coordenada(len(tablero),abj)
                    newListaCapturasDer=listaCapturas[:]
                    newListaCapturasDer.append((der,newArr))
                    newEstado=deepcopy(self)
                    newEstado.tablero[abj][der]=0
                    newEstado.tablero[i][j]=0
                    newEstado.tablero[abj-1][der+1]=jugador
                    der2=der+1
                    abj2=abj-1
                    while(der2<=7 and abj2>=0):
                        if(tablero[abj2][der2]==0):
                            newEstado.comprobar_captura_dama(abj2,der2,lista,iInicial,jInicial,newListaCapturasDer)
                            der2=der2+1
                            abj2=abj2-1
                        else:
                            break
                    break
                else:
                    break
            elif(tablero[abj][der]==0):
                der=der+1
                abj=abj-1
            else:
                break
        izq=j-1
        arr=i+1
        while(izq>=1 and arr<=6):
            if(tablero[arr][izq]==peonRival or tablero[arr][izq]==reyRival):
                if(tablero[arr+1][izq-1]==0):
                    newArr=Player.cambio_coordenada(len(tablero),arr)
                    newListaCapturasIzq=listaCapturas[:]
                    newListaCapturasIzq.append((izq,newArr))
                    newEstado=deepcopy(self)
                    newEstado.tablero[arr][izq]=0
                    newEstado.tablero[i][j]=0
                    newEstado.tablero[arr+1][izq-1]=jugador
                    arr2=arr+1
                    izq2=izq-1
                    while(izq2>=0 and arr2<=7):
                        if(tablero[arr2][izq2]==0):
                            newEstado.comprobar_captura_dama(arr2,izq2,lista,iInicial,jInicial,newListaCapturasIzq)
                            izq2=izq2-1
                            arr2=arr2+1
                        else:
                            break
                    break
                else:
                    break
            elif(tablero[arr][izq]==0):
                izq=izq-1
                arr=arr+1
            else:
                break
        izq=j-1
        abj=i-1
        while(izq>=1 and abj>=1):
            if(tablero[abj][izq]==peonRival or tablero[abj][izq]==reyRival):
                if(tablero[abj-1][izq-1]==0):
                    newArr=Player.cambio_coordenada(len(tablero),abj)
                    newListaCapturasIzq=listaCapturas[:]
                    newListaCapturasIzq.append((izq,newArr))
                    newEstado=deepcopy(self)
                    newEstado.tablero[abj][izq]=0
                    newEstado.tablero[i][j]=0
                    newEstado.tablero[abj-1][izq-1]=jugador
                    izq2=izq-1
                    abj2=abj-1
                    while(izq2>=0 and abj2>=0):
                        if(tablero[abj2][izq2]==0):
                            newEstado.comprobar_captura_dama(abj2,izq2,lista,iInicial,jInicial,newListaCapturasIzq)
                            izq2=izq2-1
                            abj2=abj2-1
                        else:
                            break
                    break
                else:
                    break
            elif(tablero[abj][izq]==0):
                izq=izq-1
                abj=abj-1
            else:
                break
        if(len(listaCapturas)>0):
            lista.append((len(listaCapturas),(jInicial,iInicial,j,newI),listaCapturas))
        return lista

    #Funcion que devuelve una lista con los movimientos que mas capturas tiene
    @staticmethod
    def limpiar_lista_capturas(lista):
        lista.sort(key=lambda l:l[0], reverse=True)
        movimientosMasCapturas=[[mov[1],mov[2]] for mov in lista if mov[0]==lista[0][0]]
        return movimientosMasCapturas

    #Esta funcion es la encargada de obtener los posible movimientos de un jugador
    def obtiene_movimientos(self):
        tablero = self.tablero
        jugador = self.jugadorActual
        jugadorRey = jugador+2
        movimientos = []
        numer_filas = len(tablero)
        movimiento=(0,0,0,0)
        captura=False
        numCapturasMax=0
        for i in range(numer_filas):
            if(jugador or jugadorRey in tablero[i]):
                for j in range(len(tablero[i])):
                    if(tablero[i][j]==jugador):
                        #Comprobamos si puede comer, si es asi esta obligado a hacer el movimiento con mas capturas
                        movimientosCaptura=self.comprobar_captura(i,j,[],Player.cambio_coordenada(numer_filas,i),j)
                        if(movimientosCaptura):
                            movimientosCaptura=self.limpiar_lista_capturas(movimientosCaptura)
                            numCapturas=len(movimientosCaptura[0][1])
                            #Si hay captura se olvidan los anteriores movimientos
                            if(captura==False):
                                movimientos=[]
                                captura=True
                            #Si la captura es mayor que las anteriores se olvidan los anteriores movimientos
                            if(numCapturas>numCapturasMax):
                                movimientos=[]
                                movimientos=movimientosCaptura
                                numCapturasMax=numCapturas
                            #Si la captura es igual que las anteriores se añaden los nuevos movimientos
                            elif(numCapturas==numCapturasMax):
                                movimientos.extend(movimientosCaptura)
                                numCapturasMax=numCapturas
                        #Si no puede comer y no hay capturas anteriores se añaden los movimientos normales
                        elif(captura==False): 
                            newI=Player.cambio_coordenada(numer_filas,i)
                            izq=j-1
                            der=j+1
                            if(jugador==1):
                                arr=i+1
                            else:
                                arr=i-1
                            newArr=Player.cambio_coordenada(numer_filas,arr)
                            if(izq>=0 and (arr<=7 and arr>=0)):
                                if(tablero[arr][izq]==0):
                                    movimiento=(j,newI,izq,newArr)
                                    movimientos.append(movimiento)                  
                            if(der<=7 and (arr<=7 and arr>=0)):
                                if(tablero[arr][der]==0):
                                    movimiento=(j,newI,der,newArr)
                                    movimientos.append(movimiento)
                    elif(tablero[i][j]==jugadorRey):
                        movimientosCapturaDama=self.comprobar_captura_dama(i,j,[],Player.cambio_coordenada(numer_filas,i),j)
                        if(movimientosCapturaDama):
                            movimientosCapturaDama=self.limpiar_lista_capturas(movimientosCapturaDama)
                            numCapturas=len(movimientosCapturaDama[0][1])
                            #Si hay captura se olvidan los anteriores movimientos
                            if(captura==False):
                                movimientos=[]
                                captura=True
                            #Si la captura es mayor que las anteriores se olvidan los anteriores movimientos
                            if(numCapturas>numCapturasMax):
                                movimientos=[]
                                movimientos=movimientosCapturaDama
                                numCapturasMax=numCapturas
                            #Si la captura es igual que las anteriores se añaden los nuevos movimientos
                            elif(numCapturas==numCapturasMax):
                                movimientos.extend(movimientosCapturaDama)
                                numCapturasMax=numCapturas
                        #Si no puede comer y no hay capturas anteriores se añaden los movimientos normales
                        elif(captura==False):   
                            izq=j-1
                            arr=i-1
                            newI=Player.cambio_coordenada(numer_filas,i)
                            newArr=Player.cambio_coordenada(numer_filas,arr)
                            while(izq>=0 and arr>=0):
                                newArr=Player.cambio_coordenada(numer_filas,arr)
                                if(tablero[arr][izq]==0):
                                    movimiento=(j,newI,izq,newArr)
                                    movimientos.append(movimiento)
                                    izq=izq-1
                                    arr=arr-1
                                else:
                                    break
                            der=j+1 
                            arr=i-1
                            newI=Player.cambio_coordenada(numer_filas,i)
                            newArr=Player.cambio_coordenada(numer_filas,arr)
                            while(der<=7 and arr>=0):
                                newArr=Player.cambio_coordenada(numer_filas,arr)
                                if(tablero[arr][der]==0):
                                    movimiento=(j,newI,der,newArr)
                                    movimientos.append(movimiento)
                                    der=der+1
                                    arr=arr-1
                                else:
                                    break
                            izq=j-1
                            abj=i+1
                            newI=Player.cambio_coordenada(numer_filas,i)
                            newAbj=Player.cambio_coordenada(numer_filas,abj)   
                            while(izq>=0 and abj<=7):
                                newAbj=Player.cambio_coordenada(numer_filas,abj)
                                if(tablero[abj][izq]==0):
                                    movimiento=(j,newI,izq,newAbj)
                                    movimientos.append(movimiento)
                                    izq=izq-1
                                    abj=abj+1
                                else:
                                    break
                            der=j+1 
                            abj=i+1
                            newI=Player.cambio_coordenada(numer_filas,i)
                            newAbj=Player.cambio_coordenada(numer_filas,abj)   
                            while(der<=7 and abj<=7):
                                newAbj=Player.cambio_coordenada(numer_filas,abj)
                                if(tablero[abj][der]==0):
                                    movimiento=(j,newI,der,newAbj)
                                    movimientos.append(movimiento)
                                    der=der+1
                                    abj=abj+1
                                else:
                                    break
        return random.sample(movimientos, len(movimientos))

    def aplica_movimiento(self,movimiento):
        tablero =self.tablero
        jugador = self.jugadorActual
        self.jugadorActual = 3 - self.jugadorActual
        self.turnos = self.turnos - 1
        if(len(movimiento)==4):
            i=movimiento[1]
            iTrad=Player.cambio_coordenada(len(tablero),i)
            j=movimiento[0]
            newI=movimiento[3]
            newITrad=Player.cambio_coordenada(len(tablero),newI)
            newJ=movimiento[2]
            ficha=tablero[iTrad][j]
            #Si la ficha llega al final del tablero se convierte en dama
            if(newI==7 or newI==0):
                if(ficha==1 or ficha==2):
                    ficha=ficha+2
            tablero[iTrad][j]=0
            tablero[newITrad][newJ]=ficha
            return self
        else:
            posicionesMovimiento=movimiento[0]
            capturas=movimiento[1]
            i=posicionesMovimiento[1]
            iTrad=Player.cambio_coordenada(len(tablero),i)
            j=posicionesMovimiento[0]
            newI=posicionesMovimiento[3]
            newITrad=Player.cambio_coordenada(len(tablero),newI)
            newJ=posicionesMovimiento[2]
            ficha=tablero[iTrad][j]
            #Si la ficha llega al final del tablero se convierte en dama
            if(newI==7 or newI==0):
                if(ficha==1 or ficha==2):
                    ficha=ficha+2
            tablero[iTrad][j]=0
            tablero[newITrad][newJ]=ficha
            for captura in capturas:
                i=captura[1]
                iTrad=Player.cambio_coordenada(len(tablero),i)
                j=captura[0]
                tablero[iTrad][j]=0
            return self

    #Comprueba si el jugador tiene alguna ficha en el tablero
    def jugador_tiene_fichas(self,jugador):
        tablero = self.tablero
        res = False
        dama=jugador+2
        for fila in tablero:
            if(jugador in fila or dama in fila):
                res = True
                break
        return res

    def gana_jugador(self,jugador):
        numeroMovimientos = len(self.obtiene_movimientos())
        if(jugador==2):
            return self.ganan_blancas(numeroMovimientos)
        else:
            return self.ganan_negras(numeroMovimientos)

    def ganan_negras(self,numeroMovimientos):
        jugador = self.jugadorActual
        resultado = False
        if(numeroMovimientos==0 and jugador==2):
            resultado = True
        elif(self.jugador_tiene_fichas(2)==False):
            resultado = True
        return resultado

    def ganan_blancas(self,numeroMovimientos):
        jugador = self.jugadorActual
        resultado = False
        if(numeroMovimientos==0 and jugador==1):
            resultado = True
        elif(self.jugador_tiene_fichas(1)==False):
            resultado = True
        return resultado

    def es_estado_final(self):
        numeroMovimientos = len(self.obtiene_movimientos())
        numeroTurnos=self.turnos
        if(numeroTurnos==0):
            return True
        else:
            blancas=self.ganan_blancas(numeroMovimientos)
            negras=self.ganan_negras(numeroMovimientos)
            return blancas or negras

    def imprime_tablero(self):
        tablero = self.tablero
        numFilas = len(tablero)
        for i in range(numFilas):
            print("\n",Player.cambio_coordenada(numFilas,i),end="   ")
            for j in range(len(tablero[i])):
                if(tablero[i][j]==1):
                    print("n",end=" ")
                elif(tablero[i][j]==2):
                    print("b",end=" ")
                elif(tablero[i][j]==3):
                    print("N",end=" ")
                elif(tablero[i][j]==4):
                    print("B",end=" ")
                else:
                    print("-",end=" ")
        print("\n")
        for i in range(numFilas):
            if (i==0):
                print("     ",end="")
            print(i,end=" ")
        print("\n")

    def imprime_estado(self):
        turnos = self.turnos
        jugador = self.jugadorActual
        self.imprime_tablero()
        print("\nTurnos restantes: ",turnos)
        if(jugador==1):
            print("\nTurno de las negras")
        else:
            print("\nTurno de las blancas")

    def imprime_final(self):
        print("\nFin del juego\n")
        numeroMovimientos = self.obtiene_movimientos()
        #Recorremos el tablero e imprimimos en consola el valor de cada casilla siendo 
        self.imprime_tablero()
        #Se comprueba si el estado es final y se imprime el resultado
        if(self.ganan_blancas(numeroMovimientos)):
                print("\nGanan blancas\n")
                return 2
        elif(self.ganan_negras(numeroMovimientos)):
                print("\nGanan negras\n")
                return 1
    
    def turno_jugador(self):
        print(Fore.RED+"\nTurno Jugador "+str(self.jugadorActual))
        self.imprime_tablero()
        movimientos = self.obtiene_movimientos()
        Player.imprime_movimientos(movimientos)
        print("Introduce el movimiento que quieres realizar (seleccionado el numero 1 o 2 o 3,..): ")
        movimiento=int(input())
        while(movimiento<=0 or movimiento>len(movimientos)):
            print("Movimiento invalido, introduce un movimiento: ")
            movimiento=int(input())
        return self.aplica_movimiento(movimientos[movimiento-1])
    
      
    def turno_alfabeta(self,alfabeta):
        print(Fore.GREEN+"\nTurno Alfabeta "+str(self.jugadorActual))
        self.imprime_tablero()
        mov = alfabeta.ejecuta(self)
        if(mov!=None):
            return self.aplica_movimiento(mov)
        else:
            return self
    


In [9]:
class AlfaBeta(Damas):
  def __init__(self, dama):
        self.aplicar_movimiento = staticmethod(Damas.aplica_movimiento)
        self.obtener_movimientos = staticmethod(Damas.obtiene_movimientos)
        self.es_estado_final = staticmethod(Damas.es_estado_final)
        self.gana_jugador = staticmethod(Damas.gana_jugador)
        self.heuristica = staticmethod(Damas.heuristica)
        self.jugadores = [i+1 for i in range(Damas.jugadores)]
        self.profundidadBusqueda = 6
        super().__init__()
        
  def evaluate(self,estado,jugadorMax):
          if(self.es_estado_final(estado)):
              for jugador in self.jugadores:
                  if(self.gana_jugador(estado,jugador)):
                      if(jugador==jugadorMax):
                          return math.inf
                      else:
                          return -math.inf
              return 0
          return self.heuristica(estado,jugadorMax)

  def alfabeta(self, estado, depth, movMax, jugadorMax, profundidadBusqueda, alpha, beta, estadosEvaluados):
        movs = self.obtener_movimientos(estado)
        score = self.evaluate(estado,jugadorMax)
        jugador=estado.jugadorActual
        newEstado=deepcopy(estado)

        def max_value(self, estado, depth, movMax, jugadorMax, profundidadBusqueda, alpha, beta, estadosEvaluados):
          if(self.es_estado_final(estado)):
                return ((score - depth), movMax, estadosEvaluados)

          if(depth<=profundidadBusqueda):
              if(jugadorMax==jugador):
                    best = -math.inf
                    for i in range(len(movs)):
                        s = self.aplicar_movimiento(newEstado,movs[i])
                        value=self.alfabeta(s, depth+1, movMax, jugadorMax, profundidadBusqueda, alpha, beta, estadosEvaluados+1)
                        if(value[0]>best):
                            movMax=movs[i]
                        best = max(best, value[0])
                        alpha = max(alpha, best)
                        newEstado=deepcopy(estado)
                        estadosEvaluados=value[2]
                        if (beta <= alpha):
                            break
                    return (best, movMax, estadosEvaluados)
        
        def min_value(self, estado, depth, movMax, jugadorMax, profundidadBusqueda, alpha, beta, estadosEvaluados):
          if(self.es_estado_final(estado)):
                return ((score - depth), movMax, estadosEvaluados)


          if(depth<=profundidadBusqueda):
              if(jugadorMax==jugador):
                  best = math.inf
                  for i in range(len(movs)):
                      s = self.aplicar_movimiento(newEstado,movs[i])
                      value=self.alfabeta(s,  depth+1, movMax, jugadorMax, profundidadBusqueda, alpha, beta, estadosEvaluados+1)
                      best = min(best,value[0])
                      beta = min(beta, best)
                      newEstado=deepcopy(estado)
                      estadosEvaluados=value[2]
                      if (beta <= alpha):
                          break
                  return (best, movMax, estadosEvaluados)
        return ((score - depth), movMax, estadosEvaluados)

  def ejecuta(self, estado):
          t0 = time.time()
          alpha=-math.inf #Maximizar
          beta=math.inf #Minimizar
          jugadorMax = estado.jugadorActual
          movs = self.obtener_movimientos(estado)
          newEstado = deepcopy(estado)
          if(len(movs)==1):
              return movs[0]
          elif(len(movs)==0):
              return None
          else:
              valores = self.alfabeta(newEstado, 0, movs[0], jugadorMax, self.profundidadBusqueda, alpha, beta, 1)
              mov= valores[1]
              estadosEvaluados = valores[2]
          return mov

In [10]:
def jugadorContraAlgoritmo():

    s = Damas()    
    algoritmo = AlfaBeta(s)
    
    posicionJugador = int(input("¿Qué ficha quieres usar? (1. Negras o 2. Blancas) \n"))
    while(posicionJugador<1 or posicionJugador>2):
        posicionJugador = int(input("Introduce 1 si quieres jugar negras o 2 si quieres jugar blancas: \n"))
    
    while(s.es_estado_final()==False):
        jugador = s.jugadorActual
        if(jugador==posicionJugador):
            s = s.turno_jugador()
        else:
            if(algoritmo.__class__.__name__=="AlfaBeta"):
                s = s.turno_alfabeta(algoritmo)   
    s.imprime_final()   


In [11]:
def main():
    print("\nBienvenido al juego de las Damas\n")
    jugadorContraAlgoritmo()

if __name__ == "__main__":
    main()


Bienvenido al juego de las Damas

¿Qué ficha quieres usar? (1. Negras o 2. Blancas) 
1

Turno Alfabeta 2

 7   - n - n - n - n 
 6   n - n - n - n - 
 5   - n - n - n - n 
 4   - - - - - - - - 
 3   - - - - - - - - 
 2   b - b - b - b - 
 1   - b - b - b - b 
 0   b - b - b - b - 

     0 1 2 3 4 5 6 7 


Turno Jugador 1

 7   - n - n - n - n 
 6   n - n - n - n - 
 5   - n - n - n - n 
 4   - - - - - - - - 
 3   - - - - - b - - 
 2   b - b - b - - - 
 1   - b - b - b - b 
 0   b - b - b - b - 

     0 1 2 3 4 5 6 7 

1 º movimiento: (1, 5, 0, 4)
2 º movimiento: (5, 5, 6, 4)
3 º movimiento: (3, 5, 4, 4)
4 º movimiento: (3, 5, 2, 4)
5 º movimiento: (5, 5, 4, 4)
6 º movimiento: (7, 5, 6, 4)
7 º movimiento: (1, 5, 2, 4)
Introduce el movimiento que quieres realizar (seleccionado el numero 1 o 2 o 3,..): 
7

Turno Alfabeta 2

 7   - n - n - n - n 
 6   n - n - n - n - 
 5   - - - n - n - n 
 4   - - n - - - - - 
 3   - - - - - b - - 
 2   b - b - b - - - 
 1   - b - b - b - b 
 0   b - b -

KeyboardInterrupt: ignored